In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from tqdm import tqdm
import time
import os

In [2]:
KEY = '0BAEAA44F5C3C9967C53E41AE9B50F25'

In [3]:
df = pd.read_csv('output_files/games.csv')
df_tmp = df[df['total_recommendations']>0]
app_ids = df_tmp['steam_appid'].tolist()

In [4]:
players = set()

In [5]:
if not os.path.exists('output_files/players.csv'):
    print("No players found.")
    with open('output_files/players.csv', 'a') as F:
        writer = csv.writer(F)
        cols = ['steam_id', ]
        writer.writerow(cols)
else:
    print("Some players found, ids loaded.")
    df_players = pd.read_csv('output_files/players.csv')
    players = set(df_players['steam_id'].tolist())

Some players found, ids loaded.


In [6]:
if not os.path.exists('output_files/resolved_appid.csv'):
    print('No resolved ids found.')
    with open('output_files/resolved_appid.csv', 'a') as F_1:
        writer_1 = csv.writer(F_1)
        cols = ['steam_appid', ]
        writer_1.writerow(cols)
else:
    print('Already resolved some games ids, removing them from the list...')
    df_resolved = pd.read_csv('output_files/resolved_appid.csv')
    resolved_app_ids = df_resolved['steam_appid'].tolist()
    app_ids = list(set(app_ids) - set(resolved_app_ids))


Already resolved some games ids, removing them from the list...


In [7]:
if not os.path.exists('output_files/error_appid.csv'):
    with open('output_files/resolved_appid.csv', 'w') as F_2:
        writer_2 = csv.writer(F_2)
        cols = ['steam_appid', ]
        writer_2.writerow(cols)

In [8]:
with open('output_files/players.csv', 'a') as F:
    for app_id in tqdm(app_ids, total=len(app_ids)):
        try:
            writer = csv.writer(F)
            url = f'https://store.steampowered.com/appreviews/{app_id}?json=1&language=all&num_per_page=100'
            response = requests.get(url)
            data = response.json()
            for review in data['reviews']:
                player_id = review['author']['steamid']
                if player_id not in players:
                    players.add(player_id)
                    writer.writerow([player_id])
            with open('output_files/resolved_appid.csv', 'a') as F_1:
                writer_1 = csv.writer(F_1)
                writer_1.writerow([app_id])
        except:
            with open('output_files/error_appid.csv', 'w') as F_2:
                writer_2 = csv.writer(F_2)
                writer_2.writerow([app_id])
        time.sleep(1)

  0%|          | 12/15807 [00:18<6:51:02,  1.56s/it]


KeyboardInterrupt: 

In [ ]:
df_players = pd.read_csv('output_files/players.csv')
steam_ids = set(df_players['steam_id'].tolist())

In [ ]:
def get_reviews(steam_id, page_number):
    url = f"https://steamcommunity.com/profiles/{steam_id}/recommended/?p={page_number}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    reviews = soup.findAll('div', {'class': 'review_box'})
    return reviews

In [ ]:
if not os.path.exists('output_files/resolved_steam_ids.csv'):
    print("No resolved steam ids found.")
    with open('output_files/resolved_steam_ids.csv', 'a') as F:
        writer = csv.writer(F)
        cols = ['steam_id', ]
        writer.writerow(cols)
else:
    print("Some resolved steam ids found, ids loaded.")
    df_resolved = pd.read_csv('output_files/resolved_steam_ids.csv')
    resolved_steam_ids = df_resolved['steam_id'].tolist()
    steam_ids = list(set(steam_ids) - set(resolved_steam_ids))


with open('output_files/game_reviews.csv', 'a') as F:
    for steam_id in tqdm(steam_ids, total=len(steam_ids)):
        try:
            i = 1
            while i!=0:
                reviews = get_reviews(steam_id, i)
                if reviews != []:
                    for review in reviews:
                        review_data = []
                        review_data.append(str(steam_id))
                        app_id = review.find('a', {'class': 'game_capsule_ctn'})['href'][31:]
                        review_data.append(app_id)
                        recommended = review.find('div', {'class': 'title'}).text
                        review_data.append(recommended)
                        hours_played = review.find('div', {'class': 'hours'}).text
                        hours_played = hours_played.replace('\r', '')
                        hours_played = hours_played.replace('\n', '')
                        hours_played = hours_played.replace('\t', '')
                        review_data.append(hours_played)
                        writer.writerow(review_data)
                    i += 1
                    time.sleep(1.5)
                else:
                    i = 0
                    with open('output_files/resolved_steam_ids.csv', 'a') as F:
                        writer = csv.writer(F)
                        writer.writerow([steam_id])
        except:
            with open('output_files/error_steam_ids.csv', 'w') as F:
                writer = csv.writer(F)
                writer.writerow([steam_id])